<a href="https://colab.research.google.com/github/mei-yong/realtime_architecture/blob/main/Delta_Live_Table_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=BIxwoO65ylY

https://www.youtube.com/watch?v=PIFL7W3DmaY

Create bronze table

In [ ]:
%sql
CREATE STREAMING LIVE TABLE table_name_raw
COMMENT "Placeholder comment here."
TBLPROPERTIES ("quality"="bronze")
AS
SELECT * FROM cloud_files("/folder/folder/file", "json", map("cloudFiles.inferColumnTypes", "true"))
--using Databricks Auto Loader
--the true in map enables capturing unexpected data in a rescue data column and restarts the pipeline to use the new data

Create silver table

In [ ]:
%sql
CREATE STREAMING LIVE TABLE table_name_clean (
  CONSTRAINT valid_id EXPECT (table_id IS NOT NULL) ON VIOLATIOIN DROP ROW,
  CONSTRAINT over_24hr_old_data EXPECT (secs_since_last_reported < 86400)
)
PARTITIONED BY (last_updated_date)
COMMENT "Placeholder comment here."
TBLPROPERTIES ("quality"="silver")
AS
SELECT
  field.table_id,
  field.other_column,
  CAST(field.last_reported AS timestamp) AS last_reported_ts,
  last_updated,
  last_updated_ts,
  date(last_updated_ts) AS last_updated_date,
  unix_timestamp(date_trunc('HOUR', last_updated_ts)) AS last_updated_hr,
  last_updated - field.last_reported AS secs_since_last_reported
FROM (
    SELECT
      EXPLORE(data.field) AS field,
      last_updated,
      CAST(last_updated AS timestamp) AS last_updated_ts
    FROM STREAM(LIVE.table_name_raw)
)

Temporary bronze table for handling slowly changing dimensions

In [ ]:
CREATE TEMPORARY STREAMING LIVE TABLE exploded_other_raw_data
AS
SELECT
  stuff.a,
  stuff.b
FROM (
  EXPLODE(data.stuff) AS stuff,
  last_updated,
  CAST(last_updated AS timestamp) AS last_updated_ts
FROM STREAM(LIVE.other_raw_data)
)

Silver table for storing the SCD table

In [ ]:
CREATE STREAMING LIVE TABLE stuff_clean
COMMENT "Placeholder comment."
TBLPROPERTIES ("quality"="silver")

In [ ]:
--Upsert/merge
APPLY CHANGES INTO LIVE.stuff_clean FROM STREAM(LIVE.exploded_other_raw_data)
  KEYS(table_id)
  SEQUENCE BY last_updated